# T6. WALL RECONSTRUCTION

In this script, we reconstruct parametric wall geometries from the instance segmentation and reference heights.
Specifically, we need:
 - T2: instances of walls, ceilings and other objects
 - T5: reference levels

## LIBRARIES

In [7]:
#IMPORT PACKAGES
from rdflib import Graph, URIRef
import os.path
import importlib
from pathlib import Path
import numpy as np
import xml.etree.ElementTree as ET
import open3d as o3d
import uuid    
import pye57 
import ifcopenshell
import ifcopenshell.geom as geom
import ifcopenshell.util
from ifcopenshell.util.selector import Selector
import multiprocessing
import random as rd
import pandas as pd
# from tabulate import tabulate
import cv2
import laspy
import json
from scipy.spatial.transform import Rotation   
import copy
import geomapi
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl

#import utils
from context import utils
import utils as utl
import utils.t1_utils as t1


In [3]:
%load_ext autoreload

In [4]:
%autoreload 2

## INPUTS

In [5]:
name='beton_labels'

#point clouds
path=Path(os.getcwd()).parents[2]/'data'
pcd_input_path=os.path.join(path,f'{name}.laz')
class_file=path/'_classes.json'

#reference levels
path_reference_levels=os.path.join(path,f'{name}.laz') #eigenlijk zou dit een graph met BIM elementen moeten zijn!

#output
name=name.split('_')[0]
json_output_path=os.path.join(path,f'{name}_walls.json') 
geometry_output_path= os.path.join(path,f'{name}_walls.obj') # these are the bounding surfaces of the reference levels (optional)


Import Classes

In [8]:
# Read the JSON file
with open(class_file, 'r') as file:
    json_data = json.load(file)

# Create a dictionary
class_dict = {
    'classes': json_data['classes'],
    'default': json_data['default'],
    'type': json_data['type'],
    'format': json_data['format'],
    'created_with': json_data['created_with']
}
print(class_dict)

{'classes': [{'name': 'Unassigned', 'id': 255, 'temp_id': -1, 'color': '#9da2ab'}, {'name': 'Floors', 'id': 0, 'temp_id': 0, 'color': '#03c2fc'}, {'name': 'Ceilings', 'id': 1, 'temp_id': 1, 'color': '#e81416'}, {'name': 'Walls', 'id': 2, 'temp_id': 2, 'color': '#ffa500'}, {'name': 'Columns', 'id': 3, 'temp_id': 3, 'color': '#faeb36'}, {'name': 'Doors', 'id': 4, 'temp_id': 4, 'color': '#79c314'}, {'name': 'Windows', 'id': 5, 'temp_id': 5, 'color': '#4b369d'}], 'default': 255, 'type': 'semantic_segmentation', 'format': 'kitti', 'created_with': {'name': 'Saiga', 'version': '1.0.1'}}


Import PCD

In [10]:
laz=laspy.read(pcd_input_path)

In [11]:
pcdNodes=[]
#split pcd per object
for i in np.unique(laz['classes']):
    idx=np.where(laz['classes']==i)
    points=laz.xyz[idx]
    # colors=np.array([laz.red[idx],laz.green[idx],laz.blue[idx]])
    object_labels=laz['objects'][idx]

    class_obj=next((class_obj for class_obj in json_data['classes'] if float(class_obj['id']) ==i), json_data['classes'][0])
    class_name=class_obj['name']

    # pcd.colors=o3d.utility.Vector3dVector(colors)
    for j in np.unique(object_labels):
        
        new_points=points[np.where(object_labels==j)]
        if new_points.shape[0]>100:
            pcd=o3d.geometry.PointCloud()
            pcd.points=o3d.utility.Vector3dVector(new_points)

            pcdNodes.append(PointCloudNode(resource=pcd,
                                        class_id=i,
                                        object_id=j,
                                        color=ut.random_color(),
                                            name=class_name+f'_{j}'))
                                            

print(f'{len(pcdNodes)} pcdNodes created!')

85 pcdNodes created!


In [18]:
joined_pcd=gmu.join_geometries([n.resource.paint_uniform_color(n.color) for n in pcdNodes if n.resource is not None])
for n in pcdNodes:
    n.orientedBoundingBox.color=[1,0,0]
o3d.visualization.draw_geometries([joined_pcd]+[n.orientedBoundingBox for n in pcdNodes])

Import Reference Levels

In [ ]:
# Read the JSON file
with open(class_file, 'r') as file:
    json_data = json.load(file)

# Create a dictionary
class_dict = {
    'classes': json_data['classes'],
    'default': json_data['default'],
}
print(class_dict)

## PROCESSING

Group wall point clouds

In [13]:
#retrieve floor points
wallNodes=[n for n in pcdNodes if n.class_id ==2]
print(f'{len(wallNodes)} wallNodes detected!')
for n in wallNodes:
    print(f' {n.name} with {n.orientedBoundingBox}')


29 wallNodes detected!
 Walls_43 with OrientedBoundingBox: center: (103775, 194659, 10.839), extent: 6.40747, 4.51928, 0.112993)
 Walls_44 with OrientedBoundingBox: center: (103780, 194669, 10.8062), extent: 3.25802, 1.29767, 0.28708)
 Walls_45 with OrientedBoundingBox: center: (103783, 194650, 12.2318), extent: 1.03244, 1.0688, 0.398084)
 Walls_46 with OrientedBoundingBox: center: (103780, 194662, 9.99442), extent: 1.69711, 0.972581, 0.181228)
 Walls_47 with OrientedBoundingBox: center: (103782, 194664, 10.1222), extent: 5.5569, 2.05446, 0.182938)
 Walls_48 with OrientedBoundingBox: center: (103785, 194673, 10.9278), extent: 3.06104, 1.51662, 0.206393)
 Walls_49 with OrientedBoundingBox: center: (103788, 194676, 11.4535), extent: 4.09731, 1.99854, 0.280056)
 Walls_50 with OrientedBoundingBox: center: (103790, 194671, 11.3065), extent: 5.00131, 4.64374, 0.179296)
 Walls_51 with OrientedBoundingBox: center: (103788, 194668, 10.3593), extent: 2.59311, 1.66859, 0.198308)
 Walls_52 with Or

Compute vertical bounding boxes

In [24]:
for n in wallNodes:

    box=n.orientedBoundingBox

    points=n._orientedBounds
    print(points)

    #compute base constraint (nearby reference level or minHeight)
    minheight= np.min(n._orientedBounds[:,2]) 
    nearby_ref_levels= tl.select_nodes_with_intersecting_bounding_box(n,referenceNodes)
    
    n.base_constraint= next(n.name for n in nearby_ref_levels)  if nearby_ref_levels else referenceNodes[0]
    n.base_offset=minheight-n.base_constraint.height
    print(n.base_constraint,n.base_offset)

    #compute top constraint
    maxHeight= np.min(n._orientedBounds[:,2])# nearby reference or ceiling level>t_max or maxHeight

    # minPoints= np.hstack((n._orientedBounds[:,0:2],n.
    # maxPoints

    # points=np.vstack((minPoints,maxPoints))


    # print(points)
    break

[[1.03772017e+05 1.94662045e+05 1.21749940e+01]
 [1.03776183e+05 1.94657479e+05 1.38630421e+01]
 [1.03772803e+05 1.94661150e+05 7.81542144e+00]
 [1.03772101e+05 1.94662121e+05 1.21744497e+01]
 [1.03777052e+05 1.94656660e+05 9.50292534e+00]
 [1.03772887e+05 1.94661226e+05 7.81487718e+00]
 [1.03776267e+05 1.94657555e+05 1.38624979e+01]
 [1.03776969e+05 1.94656584e+05 9.50346959e+00]]


NameError: name 'referenceNodes' is not defined

In [16]:
joined_pcd=gmu.join_geometries([n.resource.paint_uniform_color(n.color) for n in wallNodes if n.resource is not None])
o3d.visualization.draw_geometries([joined_pcd]+[n.orientedBoundingBox for n in wallNodes])

Split doubleWallNodes from DoubleWallNodes

In [ ]:
singleWallNodes=[]
doubleWallNodes=